In [1]:
import pandas as pd
import pickle
import os

In [2]:
def read_results_finetune(directory):
    def extract_hyperparameters(filename):
        # Remove any special characters
        parts = filename.split('_')
        dataset = parts[0]
        experiment_setup = '_'.join(parts[1:-3])
        la = float(parts[-3][2:])
        th = float(parts[-2][2:])
        bs = int(parts[-1][2:].split('.')[0])
        return dataset, experiment_setup, la, th, bs

    def load_results(directory):
        results = []
        for filename in os.listdir(directory):
            if filename.endswith('.dict'):
                filepath = os.path.join(directory, filename)
                with open(filepath, 'rb') as f:
                    data = pickle.load(f)
                dataset, experiment_setup, la, th, bs = extract_hyperparameters(filename)
                result = {
                    'dataset': dataset,
                    'experiment_setup': experiment_setup,
                    'lambda': la,
                    'theta': th,
                    'batch_size': bs,
                    'drift started at': data.get('drift started at', data.get('Drift started at')),
                    'drift detected': data.get('drift detected', data.get('Drift Detected')),
                    'data': data
                }
                results.append(result)
        return results

    results = load_results(directory)
    df = pd.DataFrame(results)
    return df

In [3]:
def read_results(directory):
    def extract_hyperparameters(filename):
        # Remove any special characters
        parts = filename.split('_')
        dataset = parts[0]
        experiment_setup = '_'.join(parts[1:3])
        return dataset, experiment_setup
    
    def load_results(directory):
        results = []
        for filename in os.listdir(directory):
            if filename.endswith('.dict'):
                filepath = os.path.join(directory, filename)
                with open(filepath, 'rb') as f:
                    data = pickle.load(f)
                dataset, experiment_setup = extract_hyperparameters(filename)
                result = {
                    'dataset': dataset,
                    'experiment_setup': experiment_setup,
                    'drift started at': data.get('drift started at', data.get('Drift started at')),
                    'drift detected': data.get('drift detected', data.get('Drift Detected')),
                    'data': data
                }
                results.append(result)
        return results

    results = load_results(directory)
    df = pd.DataFrame(results)
    return df

In [6]:
directory = 'experiments_results/IKS'
df_final_res_IKS = read_results(directory)

print(df_final_res_IKS)

  dataset  experiment_setup  drift started at drift detected  \
0   cifar   abrupt_w-0.dict             156.0             []   
1   cifar  gradual_w-0.dict             160.0             []   
2   cifar     rotate_abrupt             156.0             []   
3   cifar    rotate_gradual             160.0             []   
4   mnist   abrupt_w-0.dict             156.0             []   
5   mnist   clean_test.dict               NaN             []   
6   mnist  gradual_w-0.dict             160.0             []   
7   mnist     rotate_abrupt            5000.0             []   
8   mnist    rotate_gradual             160.0             []   

                                                data  
0  {'Drift Detected': [], 1: ([False, False, Fals...  
1  {'Drift Detected': [], 'Drift started at': 160...  
2  {'Drift Detected': [], 1: ([False, False, Fals...  
3  {'Drift Detected': [], 'Drift started at': 160...  
4  {'Drift Detected': [], 1: ([False, False, Fals...  
5  {'Drift Detected': [], 1: 

In [7]:
directory = 'experiments_results/DDAL'
df_final_res = read_results(directory)

print(df_final_res)

  dataset  experiment_setup  drift started at           drift detected  \
0   cifar   abrupt_w-0.dict              31.0      [3, 16, 28, 44, 61]   
1   cifar  gradual_w-0.dict              35.0                      [3]   
2   cifar     rotate_abrupt              31.0  [3, 16, 29, 39, 49, 59]   
3   cifar    rotate_gradual              35.0  [3, 15, 27, 41, 51, 59]   
4   mnist   abrupt_w-0.dict             156.0                    [140]   
5   mnist   clean_test.dict               NaN                    [149]   
6   mnist  gradual_w-0.dict             160.0                       []   
7   mnist     rotate_abrupt             156.0                    [118]   
8   mnist    rotate_gradual             160.0                       []   

                                                data  
0  {'Drift Detected': [3, 16, 28, 44, 61], 0: {0:...  
1  {'Drift Detected': [3], 'Drift started at': 35...  
2  {'Drift Detected': [3, 16, 29, 39, 49, 59], 0:...  
3  {'Drift Detected': [3, 15, 27, 41, 5

In [80]:
df_final_res[['dataset', 'experiment_setup', 'drift started at', 'drift detected']]

,dataset,experiment_setup,drift started at,drift detected
0,cifar,abrupt_w-0.dict,31.0,"[3, 16, 28, 44, 61]"
1,cifar,gradual_w-0.dict,35.0,[3]
2,cifar,rotate_abrupt,31.0,"[3, 16, 29, 39, 49, 59]"
3,cifar,rotate_gradual,35.0,"[3, 15, 27, 41, 51, 59]"
4,mnist,abrupt_w-0.dict,156.0,[140]
5,mnist,clean_test.dict,NaN,[149]
6,mnist,gradual_w-0.dict,160.0,[]
7,mnist,rotate_abrupt,156.0,[118]
8,mnist,rotate_gradual,160.0,[]


In [60]:
df['average_drift'] = df['drift detected'].apply(lambda x: sum(x) / len(x) if len(x) > 0 else 999)
df['drift_distance'] = df['average_drift'] - df['drift started at']

In [35]:
df[['dataset','drift started at','drift detected','experiment_setup']][(df['experiment_setup'] == 'clean_test') & (df['drift detected'].apply(lambda x: len(x) <= 1))]

,dataset,drift started at,drift detected,experiment_setup
4573,mnist,NaN,[67],clean_test
4574,mnist,NaN,[59],clean_test
4575,mnist,NaN,[],clean_test
4576,mnist,NaN,[],clean_test
4577,mnist,NaN,[],clean_test
...,...,...,...,...
5209,mnist,NaN,[],clean_test
5210,mnist,NaN,[52],clean_test
5211,mnist,NaN,[],clean_test
5212,mnist,NaN,[53],clean_test


In [37]:
df[(df['experiment_setup'] == 'clean_test') & (df['drift detected'].apply(lambda x: len(x) <= 0))]

,dataset,experiment_setup,lambda,theta,batch_size,drift started at,drift detected,data
4575,mnist,clean_test,0.825,0.825,192,NaN,[],"{'Drift Detected': [], 0: {0: 95.6521739130434..."
4576,mnist,clean_test,0.825,0.825,224,NaN,[],"{'Drift Detected': [], 0: {0: 100.0, 1: 100.0,..."
4577,mnist,clean_test,0.825,0.825,256,NaN,[],"{'Drift Detected': [], 0: {0: 95.4545454545454..."
4578,mnist,clean_test,0.825,0.825,288,NaN,[],"{'Drift Detected': [], 0: {0: 100.0, 1: 97.727..."
4580,mnist,clean_test,0.825,0.825,320,NaN,[],"{'Drift Detected': [], 0: {0: 100.0, 1: 90.909..."
...,...,...,...,...,...,...,...,...
5201,mnist,clean_test,0.900,0.975,320,NaN,[],"{'Drift Detected': [], 0: {0: 96.5517241379310..."
5207,mnist,clean_test,0.900,0.900,224,NaN,[],"{'Drift Detected': [], 0: {0: 100.0, 1: 96.774..."
5208,mnist,clean_test,0.900,0.900,256,NaN,[],"{'Drift Detected': [], 0: {0: 100.0, 1: 96.774..."
5209,mnist,clean_test,0.900,0.900,288,NaN,[],"{'Drift Detected': [], 0: {0: 93.5483870967741..."


In [64]:
df_sorted = df[(df['dataset'] == 'mnist') & (df['lambda'].between(0.825, 0.975)) & (df['theta'].between(0.825, 0.975)) & (df['drift detected'].apply(lambda x: len(x) >= 1)) & (df['experiment_setup'] != 'clean_test') & (df['drift_distance'] > -20)].sort_values('drift_distance')
df_sorted[df_sorted['drift_distance'] == 0]

,dataset,experiment_setup,lambda,theta,batch_size,drift started at,drift detected,data,average_drift,drift_distance
4264,mnist,abrupt_w-0,0.975,0.85,160,31.0,[31],"{'Drift Detected': [31], 0: {0: 92.30769230769...",31.0,0.0
4572,mnist,abrupt_w-0,0.900,0.90,96,52.0,[52],"{'Drift Detected': [52], 0: {0: 100.0, 1: 100....",52.0,0.0


In [65]:
df_sorted2 = df[(df['dataset'] == 'cifar') & (df['lambda'].between(0.825, 0.975)) & (df['theta'].between(0.825, 0.975)) & (df['drift detected'].apply(lambda x: len(x) >= 1)) & (df['experiment_setup'] != 'clean_test') & (df['drift_distance'] > -20)].sort_values('drift_distance')
df_sorted2[df_sorted2['drift_distance'] == 0]

,dataset,experiment_setup,lambda,theta,batch_size,drift started at,drift detected,data,average_drift,drift_distance
680,cifar,rotate_abrupt_rotate,0.950,0.950,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
750,cifar,rotate_abrupt_rotate,0.900,0.925,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
740,cifar,rotate_abrupt_rotate,0.975,0.900,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
630,cifar,rotate_abrupt_rotate,0.925,0.925,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
760,cifar,rotate_abrupt_rotate,0.900,0.950,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
770,cifar,rotate_abrupt_rotate,0.900,0.975,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
780,cifar,rotate_abrupt_rotate,0.900,0.900,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
640,cifar,rotate_abrupt_rotate,0.925,0.950,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
730,cifar,rotate_abrupt_rotate,0.975,0.975,256,20.0,"[1, 9, 17, 24, 31, 38]","{'Drift Detected': [1, 9, 17, 24, 31, 38], 0: ...",20.0,0.0
951,cifar,rotate_gradual_rotate,0.900,0.925,64,82.0,"[1, 29, 58, 87, 114, 134, 151]","{'Drift Detected': [1, 29, 58, 87, 114, 134, 1...",82.0,0.0


In [ ]:
df[ (df['dataset'] == 'mnist') & (df['lambda'].between(0.825, 0.975)) & (df['theta'].between(0.825, 0.975)) & (df['drift detected'].apply(lambda x: len(x) >= 1))]

In [46]:
df[(df['experiment_setup'] == 'clean_test') & (df['drift detected'].apply(lambda x: len(x) <= 4)) & (df['dataset'] == 'cifar')]

,dataset,experiment_setup,lambda,theta,batch_size,drift started at,drift detected,data
201,cifar,clean_test,0.850,0.750,288,NaN,"[5, 13, 20, 28]","{'Drift Detected': [5, 13, 20, 28], 0: {0: 76...."
203,cifar,clean_test,0.850,0.750,320,NaN,"[5, 12, 19, 26]","{'Drift Detected': [5, 12, 19, 26], 0: {0: 76...."
211,cifar,clean_test,0.850,0.775,288,NaN,"[5, 13, 20, 28]","{'Drift Detected': [5, 13, 20, 28], 0: {0: 76...."
213,cifar,clean_test,0.850,0.775,320,NaN,"[5, 12, 19, 26]","{'Drift Detected': [5, 12, 19, 26], 0: {0: 60...."
228,cifar,clean_test,0.850,0.800,288,NaN,"[5, 13, 21, 28]","{'Drift Detected': [5, 13, 21, 28], 0: {0: 61...."
230,cifar,clean_test,0.850,0.800,320,NaN,"[5, 12, 19, 26]","{'Drift Detected': [5, 12, 19, 26], 0: {0: 57...."
240,cifar,clean_test,0.925,0.925,320,NaN,"[4, 11, 18, 25]","{'Drift Detected': [4, 11, 18, 25], 0: {0: 77...."
248,cifar,clean_test,0.925,0.950,288,NaN,"[4, 12, 20, 28]","{'Drift Detected': [4, 12, 20, 28], 0: {0: 82...."
250,cifar,clean_test,0.925,0.950,320,NaN,"[4, 11, 18, 25]","{'Drift Detected': [4, 11, 18, 25], 0: {0: 68...."
260,cifar,clean_test,0.925,0.975,320,NaN,"[4, 11, 18, 25]","{'Drift Detected': [4, 11, 18, 25], 0: {0: 64...."


In [ ]:
df[['drift started at','drift detected','experiment_setup']][df['dataset'] == 'cifar']

In [ ]:
df[['drift started at','dataset','experiment_setup']][df['dataset'] == 'cifar']